In [83]:
import numpy as np
import cv2
import glob
import os


def Convert_Image(src_img, threshold_i=15):
    gray_img = np.zeros((src_img.shape[0], src_img.shape[1]), dtype=np.uint8)
    mtb_img = np.zeros((src_img.shape[0], src_img.shape[1]), dtype=np.uint8)
    mask_img = np.zeros((src_img.shape[0], src_img.shape[1]), dtype=np.uint8)
    denoise_img = np.zeros((src_img.shape[0], src_img.shape[1]), dtype=np.uint8)

    # convert to grayscale
    temp_img = src_img.astype(np.int32)    # prevent overflow
    gray_img = (temp_img[:, :, 0]*19/256 + temp_img[:, :, 1]
                * 183/256 + temp_img[:, :, 2]*54/256)     # BGR
    med_i = np.median(gray_img)

    # convert to MTB
    mtb_img = np.where(gray_img > med_i, 255, 0)

    # exclusion map
    upper_bound = med_i + threshold_i
    lower_bound = med_i - threshold_i
    mask_img[gray_img < lower_bound] = 255
    mask_img[gray_img > upper_bound] = 255

    # bind mask_img and mtb_img
    denoise_img = np.bitwise_and(mtb_img, mask_img)
    
    
    return denoise_img

# shift tar to find current best dx, dy in 9 pixels
def Shift_Image(src, tar, last_dx, last_dy):
    # convert to grayscale first (denoise_img)
    src_dimg = Convert_Image(src)
    tar_dimg = Convert_Image(tar)

    # shift last dx,dy
    h, w = src_dimg.shape[:2]
    min = h * w
    M = np.float32([[1, 0, last_dx*2], [0, 1, last_dy*2]])
    src_dimg = src_dimg.astype(np.uint8)
    tar_dimg = tar_dimg.astype(np.uint8)
    new_tar = cv2.warpAffine(tar_dimg, M, (w, h))

    # then find current best dx, dy
    for x in range(-1,2):
        for y in range(-1,2):
            M = np.float32([[1, 0, x], [0, 1, y]])        # M為平移矩陣,x為寬移動的距離,y為高
            tmp_tar = cv2.warpAffine(new_tar, M, (w, h))  # 仿射變換函式   (w, h):平移後圖像的大小
            z = np.sum(np.logical_xor(src_dimg, tmp_tar) == 1)
            if z < min:
                min = z
                dx = x
                dy = y

    # image * 2, so shift * 2
    return dx + last_dx*2, dy + last_dy*2

# input RGB image src, tar    num = scale 1/2 times
def Image_Alignment(src, tar, num):
    if num == 0:
        dx, dy = Shift_Image(src, tar, 0, 0)
    else:
        h, w = src.shape[:2]
        h_src = cv2.resize(src, (h//2,w//2))
        h_tar = cv2.resize(tar, (h//2,w//2))
        last_dx, last_dy = Image_Alignment(h_src, h_tar, num-1)
        dx, dy = Shift_Image(src, tar, last_dx, last_dy)

    return dx, dy  


# test
# a = cv2.imread('data/my_1.jpg')
# b = cv2.imread('data/my_2.jpg')

# d_x, d_y = Image_Alignment(a, b, 5)

# print(d_x,d_y)

# h, w = b.shape[:2]
# M = np.float32([[1, 0, d_x], [0, 1, d_y]])    # M為平移矩陣,x為寬移動的距離,y為高
# tmp_tar = cv2.warpAffine(b, M, (w, h))        # 仿射變換函式   (w, h):平移後圖像的大小

# cv2.imwrite('shift_img.jpg', tmp_tar)


# main()

# Read file
imgspath = glob.glob(os.path.join('data','*.png'))
imgs = [cv2.imread(i) for i in imgspath]
All_Img = []

source_image = imgs[0]
for i,ele in enumerate(imgs[1:]):
    d_x, d_y = Image_Alignment(source_image, ele, 5)
    print(d_x,d_y)

    h, w = ele.shape[:2]
    M = np.float32([[1, 0, d_x], [0, 1, d_y]])
    after_shift_ele = cv2.warpAffine(ele, M, (w, h))    

    cv2.imwrite(os.path.join('output_test', f'img{i}.png'), after_shift_ele)


# cv2.imwrite('temp.jpg', source_image)
# cv2.imwrite(os.path.join('output', f'img{i}.jpg'), myimage.denoise_img)


0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
-1 1
-1 1
-6 -2
-9 -3
-39 9
-43 11


In [84]:
import numpy as np
import cv2
import glob
import os


def Convert_Image(src_img, threshold_i=15):
    gray_img = np.zeros((src_img.shape[0], src_img.shape[1]), dtype=np.uint8)
    mtb_img = np.zeros((src_img.shape[0], src_img.shape[1]), dtype=np.uint8)
    mask_img = np.zeros((src_img.shape[0], src_img.shape[1]), dtype=np.uint8)
    denoise_img = np.zeros((src_img.shape[0], src_img.shape[1]), dtype=np.uint8)

    # convert to grayscale
    temp_img = src_img.astype(np.int32)    # prevent overflow
    gray_img = (temp_img[:, :, 0]*19/256 + temp_img[:, :, 1]
                * 183/256 + temp_img[:, :, 2]*54/256)     # BGR
    med_i = np.median(gray_img)

    # convert to MTB
    mtb_img = np.where(gray_img > med_i, 255, 0)

    # exclusion map
    upper_bound = med_i + threshold_i
    lower_bound = med_i - threshold_i
    mask_img[gray_img < lower_bound] = 255
    mask_img[gray_img > upper_bound] = 255

    # bind mask_img and mtb_img
    denoise_img = np.bitwise_and(mtb_img, mask_img)
    
    
    return denoise_img

# shift tar to find current best dx, dy in 9 pixels
def Shift_Image(src, tar, last_dx, last_dy):
    # convert to grayscale first (denoise_img)
    src_dimg = Convert_Image(src)
    tar_dimg = Convert_Image(tar)

    # shift last dx,dy
    h, w = src_dimg.shape[:2]
    min = h * w
    M = np.float32([[1, 0, last_dx*2], [0, 1, last_dy*2]])
    src_dimg = src_dimg.astype(np.uint8)
    tar_dimg = tar_dimg.astype(np.uint8)
    new_tar = cv2.warpAffine(tar_dimg, M, (w, h))

    # then find current best dx, dy
    for x in range(-1,2):
        for y in range(-1,2):
            M = np.float32([[1, 0, x], [0, 1, y]])        # M為平移矩陣,x為寬移動的距離,y為高
            tmp_tar = cv2.warpAffine(new_tar, M, (w, h))  # 仿射變換函式   (w, h):平移後圖像的大小
            z = np.sum(np.logical_xor(src_dimg, tmp_tar) == 1)
            if z < min:
                min = z
                dx = x
                dy = y

    # image * 2, so shift * 2
    return dx + last_dx*2, dy + last_dy*2

# input RGB image src, tar    num = scale 1/2 times
def Image_Alignment(src, tar, num):
    if num == 0:
        dx, dy = Shift_Image(src, tar, 0, 0)
    else:
        h, w = src.shape[:2]
        h_src = cv2.resize(src, (h//2,w//2))
        h_tar = cv2.resize(tar, (h//2,w//2))
        last_dx, last_dy = Image_Alignment(h_src, h_tar, num-1)
        dx, dy = Shift_Image(src, tar, last_dx, last_dy)

    return dx, dy  


imgspath = glob.glob(os.path.join('data','*.png'))
imgs = [cv2.imread(i) for i in imgspath]
All_Img = []

# source_image = imgs[0]
for i,ele in enumerate(imgs[0:]):
    outpu = Convert_Image(ele)
    # print(d_x,d_y)

    # h, w = ele.shape[:2]
    # M = np.float32([[1, 0, d_x], [0, 1, d_y]])
    # after_shift_ele = cv2.warpAffine(ele, M, (w, h))    

    cv2.imwrite(os.path.join('output_mask', f'img{i}.png'), outpu)